# Testing S-57 Data Processing Classes

# S-57 Data Conversion Workflows

This notebook demonstrates comprehensive S-57 (Electronic Navigational Chart) conversion workflows
using the `nautical_graph_toolkit` library. Learn how to convert raw S-57 files to multiple backend formats
and verify data integrity across the conversion pipeline.

## Conversion Modes

### 1. **by_enc Mode** (S57Base)
- Each S-57 file becomes a separate output database/schema
- Useful for: Managing individual chart versions, isolated testing, source tracking
- Output structure: One schema per ENC (e.g., `US1EEZ1M`, `US1GC09M`)

### 2. **by_layer Mode** (S57Advanced) - RECOMMENDED
- All S-57 files merged into a single database organized by object class (layer)
- Useful for: Unified analysis, spatial queries, performance optimization
- Features are stamped with `dsid_*` columns tracking source ENC, edition, and update
- Better for: Routing, maritime analysis, data fusion

## Backend Targets
- **PostGIS**: Multi-user, server-based, enterprise-ready
- **GeoPackage**: Portable single file, works offline
- **SpatiaLite**: Self-contained SQLite, no external dependencies

## Performance Considerations
- S57Advanced with `auto_tune_batch_size=True`: Automatically optimizes memory and I/O
- Parallel processing: Available for read-only file discovery (not database writes)
- Large datasets (10000+ ENCs): May take 2-4 hours depending on hardware


## 1. Environment Setup

Initialize the Python environment and import required modules. This section:
- Adds the project root to Python path for module imports
- Loads environment variables from `.env` file
- Imports data conversion classes
- Defines data paths and database connection parameters

**Important**: Ensure your `.env` file contains:
- `DB_NAME`, `DB_USER`, `DB_PASSWORD`, `DB_HOST`, `DB_PORT` for PostGIS
- Or comment out PostGIS sections if using file-based backends only


In [1]:
import sys
import os
from distutils.command.config import config
from pathlib import Path
from dotenv import load_dotenv
# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")

from src.nautical_graph_toolkit.core.s57_data import S57Base, S57Advanced, S57Updater, PostGISManager, SpatiaLiteManager, GPKGManager, S57AdvancedConfig
# Define paths for data and output
active_enc = project_root / 'data' / 'All_ENCs' / 'ENC_ROOT'
s57_data_dir = project_root / 'data' / 'ENC_ROOT'
s57_data_update_dir = project_root / 'data' / 'ENC_ROOT_UPDATE'
output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# Define database parameters from environment variables
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

print(f"S-57 data directory: {s57_data_dir}")
print(f"Output directory: {output_dir}")
print(f"PostGIS DB_name: {db_params['dbname']}")

/tmp/ipykernel_71694/3031496437.py:3: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.command.config import config


S-57 data directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/data/ENC_ROOT
Output directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output
PostGIS DB_name: ENC_db


In [2]:
from src.nautical_graph_toolkit.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
print(pg.get_schemas())

2025-09-16 20:00:27,580 - src.maritime_module.utils.db_utils - INFO - Successfully connected to database 'ENC_db' for schema management.
['public', 's57_advanced']


In [ ]:
from osgeo import gdal
print("GDAL Python bindings version:", gdal.__version__)
print("GDAL C library version:", gdal.VersionInfo("RELEASE_NAME"))

## 2. S57Base: Simple Bulk Conversion (by_enc mode)

`S57Base` is the simplest conversion class, creating one output per input S-57 file.

**Use Cases**:
- Initial ENC assessment and validation
- Isolated testing of specific charts
- Preserving individual ENC versions unchanged
- Quick conversions of small datasets

**Conversion Process**:
1. Scans input directory for `.000` files (S-57 base files)
2. Uses GDAL's `VectorTranslate` for one-to-one mapping
3. Creates separate output per chart

**Limitation**: No feature stamping or cross-chart analysis capability


#### Convert to GPKG

In [ ]:
# import logging
# logging.getLogger('src.nautical_graph_toolkit.core.s57_data').setLevel(logging.DEBUG)

base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=str(output_dir / 'by_enc'),
        output_format='gpkg',
        overwrite=True
    )
base_converter.convert_by_enc()

#### Convert to PostGIS

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=db_params,
        output_format='postgis',
        overwrite=False
    )
base_converter.convert_by_enc()

In [ ]:
from src.nautical_graph_toolkit.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
pg.get_schema_summary()

#### Convert to SpatiaLite

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
         output_dest=str(output_dir / 'by_enc'),
        output_format='spatialite',
        overwrite=True
    )
base_converter.convert_by_enc()

## 3. S57Advanced: Optimized Layer-Centric Conversion (by_layer mode) - RECOMMENDED

`S57Advanced` performs optimized, layer-centric conversion merging all inputs into unified database.

**Key Features**:
- **Layer-centric organization**: All data organized by S-57 object class (e.g., all soundings together)
- **Feature stamping**: Each feature tagged with source ENC (`dsid_dsnm`), edition (`dsid_edtn`), update (`dsid_updn`)
- **Batch processing**: Optimized memory management for large datasets
- **Parallel processing**: Optional multi-threaded file discovery (read-only)
- **Auto-tuning**: Automatically adjusts batch size based on available memory

**Use Cases**:
- Production data integration: Build comprehensive maritime databases
- Spatial analysis: Query all features of a type across all charts
- Routing and pathfinding: Need unified navigable area representation
- Data fusion: Combine S-57 with other maritime data sources

**Configuration Options**:
- `auto_tune_batch_size`: Let system optimize performance automatically
- `enable_parallel_processing`: Use multiple cores for file discovery
- `parallel_validation_level`: Balance speed vs. data quality checks


In [3]:
# --- Step 1: Conversion ---
# NOAA ENC conversion - 2h 24m
pg_schema = 'us_enc_all' # Schema name
print(f"--- Starting S57Advanced conversion to PostGIS schema: '{pg_schema}' ---")

advanced_converter_pg = S57Advanced(
    input_path=active_enc, # Change ENC ROOT Folder Path
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=pg_schema,
    config=S57AdvancedConfig(auto_tune_batch_size=True, enable_debug_logging=False)
)
advanced_converter_pg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=pg_schema)

    # Verify a common layer exists and has data
    depare_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(depare_layer)} features from LNDMRK layer.")
    print("LNDMRK layer head (note the 'dsid_*' stamping columns):")
    display(depare_layer.head())

    # Verify feature stamping integrity across all layers
    print("\nVerifying feature update status against DSID table...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during PostGIS verification: {e}")


--- Starting S57Advanced conversion to PostGIS schema: 'us_enc_all' ---
2025-09-16 20:02:30,239 - src.maritime_module.core.s57_data - INFO - Found 6369 S-57 file(s).
2025-09-16 20:02:30,240 - src.maritime_module.core.s57_data - INFO - --- Starting optimized 'by_layer' conversion ---
2025-09-16 20:02:30,240 - src.maritime_module.core.s57_data - INFO - Files to process: 6369, Batch size: 25
2025-09-16 20:02:30,241 - src.maritime_module.core.s57_data - INFO - Pre-processing files to extract schemas and ENC names...
2025-09-16 20:03:05,368 - src.maritime_module.core.s57_data - INFO - Built unified schemas for 152 layers
2025-09-16 20:03:05,369 - src.maritime_module.utils.db_utils - INFO - Successfully connected to database 'ENC_db' for schema management.
2025-09-16 20:03:05,379 - src.maritime_module.utils.db_utils - WARNING - Overwrite is enabled. Dropping and recreating schema 'us_enc_all'...
2025-09-16 20:03:05,385 - src.maritime_module.utils.db_utils - INFO - Schema 'us_enc_all' is read

Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=274080553, FIDS=3499.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=679180273, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=851280904, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=152685725, FIDS=8527.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=970649913, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=562537632, FIDS=5624.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=161900156, FIDS=55

2025-09-16 20:42:30,739 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDARE' with 5375 files
2025-09-16 20:42:30,741 - src.maritime_module.core.s57_data - INFO - Processing layer: LNDRGN
2025-09-16 20:45:32,787 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDRGN' with 4641 files
2025-09-16 20:45:32,789 - src.maritime_module.core.s57_data - INFO - Processing layer: LNDMRK
2025-09-16 20:47:00,134 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDMRK' with 2923 files
2025-09-16 20:47:00,135 - src.maritime_module.core.s57_data - INFO - Processing layer: MAGVAR
2025-09-16 20:50:06,852 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'MAGVAR' with 6347 files
2025-09-16 20:50:06,853 - src.maritime_module.core.s57_data - INFO - Processing layer: OBSTRN
2025-09-16 20:52:53,789 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'OBSTRN' with 4674 

Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=919609508, FIDS=8659.
Skipping attribute. No more warnings will be issued.


2025-09-16 21:36:41,948 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_NPUB' with 6369 files
2025-09-16 21:36:41,950 - src.maritime_module.core.s57_data - INFO - Processing layer: M_NSYS
2025-09-16 21:40:28,212 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_NSYS' with 6369 files
2025-09-16 21:40:28,214 - src.maritime_module.core.s57_data - INFO - Processing layer: M_QUAL
2025-09-16 21:44:21,446 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_QUAL' with 6369 files
2025-09-16 21:44:21,447 - src.maritime_module.core.s57_data - INFO - Processing layer: ADMARE
2025-09-16 21:45:21,067 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'ADMARE' with 1930 files
2025-09-16 21:45:21,068 - src.maritime_module.core.s57_data - INFO - Processing layer: AIRARE
2025-09-16 21:45:35,023 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'AIRARE' with 454 f

...

Warning 1: Geometry to be inserted is of type Line String, whereas the layer geometry type is Polygon.
Insertion is likely to fail


2025-09-16 22:26:43,586 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'FSHGRD' with 5 files
2025-09-16 22:26:43,587 - src.maritime_module.core.s57_data - INFO - Processing layer: RCTLPT
2025-09-16 22:26:43,805 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'RCTLPT' with 5 files
2025-09-16 22:26:43,805 - src.maritime_module.core.s57_data - INFO - Processing layer: LOKBSN
2025-09-16 22:26:44,172 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LOKBSN' with 10 files
2025-09-16 22:26:44,173 - src.maritime_module.core.s57_data - INFO - Processing layer: RSCSTA
2025-09-16 22:26:44,268 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'RSCSTA' with 2 files
2025-09-16 22:26:44,268 - src.maritime_module.core.s57_data - INFO - Processing layer: ISTZNE
2025-09-16 22:26:44,408 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'ISTZNE' with 3 files
2025-09-

,ogc_fid,rcid,prim,grup,objl,rver,agen,fidn,fids,lnam,...,scamin,txtdsc,recdat,recind,sordat,sorind,dsid_dsnm,dsid_edtn,dsid_updn,wkb_geometry
0,1,249.0,1.0,2.0,74.0,1.0,550.0,-1.112404e+09,11426.0,0226BDB20F152CA2,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.63449 27.33063)
1,2,250.0,1.0,2.0,74.0,1.0,550.0,2.001622e+09,11426.0,0226774E52822CA2,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.56018 27.33938)
2,3,251.0,1.0,2.0,74.0,1.0,550.0,4.337476e+08,11426.0,022619DA76A62CA2,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.69787 27.38377)
3,4,252.0,1.0,2.0,74.0,1.0,550.0,-1.568839e+08,11426.0,0226F6A624602CA2,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.65553 27.34236)
4,5,253.0,1.0,2.0,74.0,1.0,550.0,1.916396e+09,11426.0,02267239E25A2CA2,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.62408 27.30319)



Verifying feature update status against DSID table...
2025-09-16 22:27:17,467 - src.maritime_module.core.s57_data - INFO - Feature update status verification complete: {'VALID': np.int64(185957)}
Verification Summary:


,layer_name,enc_name,dsid_edition,dsid_update,feature_edition,feature_update,feature_count,edition_match,update_match,status
132484,achare,US1GC09M,73,0,73,0,1,True,True,VALID
132029,achare,US2AK20M,17,1,17,1,2,True,True,VALID
132163,achare,US2AK5FM,28,0,28,0,1,True,True,VALID
132708,achare,US2EC01M,31,0,31,0,3,True,True,VALID
132747,achare,US3AK21M,21,3,21,3,1,True,True,VALID
...,...,...,...,...,...,...,...,...,...,...
122961,wrecks,US6OH05M,6,3,6,3,8,True,True,VALID
122433,wrecks,US6OH07M,4,0,4,0,1,True,True,VALID
122964,wrecks,US6OH1AD,1,0,1,0,1,True,True,VALID
120976,wrecks,US6SP10M,4,0,4,0,2,True,True,VALID


### 3.2. Convert to GeoPackage and Verify

Next, we test the conversion to a single GeoPackage file. The `GPKGManager` is used for verification.

In [2]:
# --- Step 1: Conversion ---
# NOAA ENC conversion - 4h 9m
gpkg_path = output_dir / 'us_enc_all.gpkg'
print(f"--- Starting S57Advanced conversion to GeoPackage: '{gpkg_path}' ---")

advanced_converter_gpkg = S57Advanced(
    input_path=active_enc, # Change ENC ROOT Folder Path
    output_dest=str(gpkg_path),
    output_format='gpkg',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_gpkg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = GPKGManager(gpkg_path=gpkg_path)

    # Verify a common layer exists and has data
    soundg_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(soundg_layer)} features from LNDMRK layer in the GeoPackage.")
    print("LNDMRK layer head:")
    display(soundg_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during GeoPackage verification: {e}")


--- Starting S57Advanced conversion to GeoPackage: '/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/us_enc_all.gpkg' ---
2025-09-17 09:26:31,508 - src.maritime_module.core.s57_data - INFO - Found 6369 S-57 file(s).
2025-09-17 09:26:31,509 - src.maritime_module.core.s57_data - INFO - --- Starting optimized 'by_layer' conversion ---
2025-09-17 09:26:31,509 - src.maritime_module.core.s57_data - INFO - Files to process: 6369, Batch size: 25
2025-09-17 09:26:31,510 - src.maritime_module.core.s57_data - INFO - Pre-processing files to extract schemas and ENC names...
2025-09-17 09:27:20,355 - src.maritime_module.core.s57_data - INFO - Built unified schemas for 152 layers
2025-09-17 09:27:20,356 - src.maritime_module.utils.db_utils - INFO - Output destination '/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/us_enc_all.gpkg' is ready.
2025-09-17 09:27:20,357 - src.maritime_module.core.s57_data - INFO - Processing layer: DSID


Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation opti

2025-09-17 09:34:08,263 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'DSID' with 6369 files
2025-09-17 09:34:08,265 - src.maritime_module.core.s57_data - INFO - Processing layer: IsolatedNode
2025-09-17 09:34:21,961 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'IsolatedNode' with 6369 files
2025-09-17 09:34:21,962 - src.maritime_module.core.s57_data - INFO - Processing layer: ConnectedNode
2025-09-17 09:34:37,327 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'ConnectedNode' with 6369 files
2025-09-17 09:34:37,329 - src.maritime_module.core.s57_data - INFO - Processing layer: Edge
2025-09-17 09:34:52,960 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'Edge' with 6369 files
2025-09-17 09:34:52,962 - src.maritime_module.core.s57_data - INFO - Processing layer: Face
2025-09-17 09:35:09,792 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer '

,RCID,PRIM,GRUP,OBJL,RVER,AGEN,FIDN,FIDS,LNAM,LNAM_REFS,...,SCAMIN,TXTDSC,RECDAT,RECIND,SORDAT,SORIND,DSID_DSNM,DSID_EDTN,DSID_UPDN,geometry
0,249,1,2,74,1,550,-1112404203,11426,0226BDB20F152CA2,None,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.63449 27.33063)
1,250,1,2,74,1,550,2001621634,11426,0226774E52822CA2,None,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.56018 27.33938)
2,251,1,2,74,1,550,433747622,11426,022619DA76A62CA2,None,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.69787 27.38377)
3,252,1,2,74,1,550,-156883872,11426,0226F6A624602CA2,None,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.65553 27.34236)
4,253,1,2,74,1,550,1916396122,11426,02267239E25A2CA2,None,...,119999,None,None,None,20141016,"US,US,graph,GC-11483",US4TX1GA,1,0,POINT (-97.62408 27.30319)



Verifying feature update status...
2025-09-17 13:36:09,993 - src.maritime_module.core.s57_data - INFO - Feature update status verification complete: {'VALID': np.int64(185969)}
Verification Summary:


,layer_name,enc_name,dsid_edition,dsid_update,feature_edition,feature_update,feature_count,edition_match,update_match,status
0,achare,US1GC09M,73,0,73,0,1,True,True,VALID
1,achare,US2AK20M,17,1,17,1,2,True,True,VALID
2,achare,US2AK5FM,28,0,28,0,1,True,True,VALID
3,achare,US2EC01M,31,0,31,0,3,True,True,VALID
4,achare,US3AK21M,21,3,21,3,1,True,True,VALID
...,...,...,...,...,...,...,...,...,...,...
185964,wrecks,US6OH05M,6,3,6,3,8,True,True,VALID
185965,wrecks,US6OH07M,4,0,4,0,1,True,True,VALID
185966,wrecks,US6OH1AD,1,0,1,0,1,True,True,VALID
185967,wrecks,US6SP10M,4,0,4,0,2,True,True,VALID


### 3.3. Convert to SpatiaLite and Verify

Finally, we test the conversion to a SpatiaLite database file and verify it with `SpatiaLiteManager`.

In [2]:
# --- Step 1: Conversion ---
# NOAA ENC conversion - 1h 1m
sqlite_path = output_dir / 'us_enc_all.sqlite'
print(f"--- Starting S57Advanced conversion to SpatiaLite: '{sqlite_path}' ---")

advanced_converter_sqlite = S57Advanced(
    input_path=active_enc,
    output_dest=str(sqlite_path),
    output_format='spatialite',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_sqlite.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = SpatiaLiteManager(db_path=sqlite_path)

    # Verify a common layer exists and has data
    boyspp_layer = manager.get_layer('boyspp')
    print(f"Loaded {len(boyspp_layer)} features from BOYSPP layer in the SpatiaLite DB.")
    print("BOYSPP layer head:")
    display(boyspp_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during SpatiaLite verification: {e}")


--- Starting S57Advanced conversion to SpatiaLite: '/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/us_enc_all.sqlite' ---
2025-09-22 19:49:47,425 - src.maritime_module.core.s57_data - INFO - Found 6369 S-57 file(s).
2025-09-22 19:49:47,425 - src.maritime_module.core.s57_data - INFO - --- Starting optimized 'by_layer' conversion ---
2025-09-22 19:49:47,426 - src.maritime_module.core.s57_data - INFO - Files to process: 6369, Batch size: 25
2025-09-22 19:49:47,426 - src.maritime_module.core.s57_data - INFO - Pre-processing files to extract schemas and ENC names...
2025-09-22 19:50:29,022 - src.maritime_module.core.s57_data - INFO - Built unified schemas for 152 layers
2025-09-22 19:50:29,024 - src.maritime_module.utils.db_utils - INFO - Output destination '/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/us_enc_all.sqlite' is ready.
2025-09-22 19:50:29,024 - src.maritime_module.core.s57_data - INFO - Processing layer: DSID
20

Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=274080553, FIDS=3499.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=679180273, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=851280904, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=152685725, FIDS=8527.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=970649913, FIDS=8683.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=562537632, FIDS=5624.
Skipping attribute. No more warnings will be issued.
Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=161900156, FIDS=55

2025-09-22 20:03:12,535 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDARE' with 5375 files
2025-09-22 20:03:12,537 - src.maritime_module.core.s57_data - INFO - Processing layer: LNDRGN
2025-09-22 20:04:32,065 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDRGN' with 4641 files
2025-09-22 20:04:32,066 - src.maritime_module.core.s57_data - INFO - Processing layer: LNDMRK
2025-09-22 20:05:09,285 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDMRK' with 2923 files
2025-09-22 20:05:09,286 - src.maritime_module.core.s57_data - INFO - Processing layer: MAGVAR
2025-09-22 20:06:34,117 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'MAGVAR' with 6347 files
2025-09-22 20:06:34,118 - src.maritime_module.core.s57_data - INFO - Processing layer: OBSTRN
2025-09-22 20:08:22,397 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'OBSTRN' with 4674 

Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=919609508, FIDS=8659.
Skipping attribute. No more warnings will be issued.


2025-09-22 20:27:00,344 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_NPUB' with 6369 files
2025-09-22 20:27:00,346 - src.maritime_module.core.s57_data - INFO - Processing layer: M_NSYS
2025-09-22 20:28:49,887 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_NSYS' with 6369 files
2025-09-22 20:28:49,888 - src.maritime_module.core.s57_data - INFO - Processing layer: M_QUAL
2025-09-22 20:31:09,151 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'M_QUAL' with 6369 files
2025-09-22 20:31:09,153 - src.maritime_module.core.s57_data - INFO - Processing layer: ADMARE
2025-09-22 20:31:35,957 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'ADMARE' with 1930 files
2025-09-22 20:31:35,958 - src.maritime_module.core.s57_data - INFO - Processing layer: AIRARE
2025-09-22 20:31:41,710 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'AIRARE' with 454 f

...

Warning 1: Illegal feature attribute id (ATTF:ATTL[0]) of 0
on feature FIDN=58974873, FIDS=12345.
Skipping attribute. No more warnings will be issued.


2025-09-22 20:49:33,213 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'DAMCON' with 361 files
2025-09-22 20:49:33,214 - src.maritime_module.core.s57_data - INFO - Processing layer: BOYISD
2025-09-22 20:49:34,037 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'BOYISD' with 61 files
2025-09-22 20:49:34,038 - src.maritime_module.core.s57_data - INFO - Processing layer: FSHFAC
2025-09-22 20:49:36,398 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'FSHFAC' with 166 files
2025-09-22 20:49:36,399 - src.maritime_module.core.s57_data - INFO - Processing layer: MIPARE
2025-09-22 20:49:42,616 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'MIPARE' with 383 files
2025-09-22 20:49:42,617 - src.maritime_module.core.s57_data - INFO - Processing layer: PILBOP
2025-09-22 20:49:46,623 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'PILBOP' with 300 files


,rcid,prim,grup,objl,rver,agen,fidn,fids,lnam,lnam_refs,...,scamin,txtdsc,recdat,recind,sordat,sorind,dsid_dsnm,dsid_edtn,dsid_updn,geometry
0,217,1,2,19,1,550,407554842,8676,0226184ACB1A21E4,[02261A582D5D21E4],...,119999,None,None,None,20230517,"US,US,reprt,8thCGD,LNM 20/23",US4TX1MJ,4,3,POINT (-95.03855 29.18491)
1,218,1,2,19,1,550,316468004,4025,022612DCEB240FB9,None,...,119999,None,None,None,20060124,"US,US,graph,Chart 11322",US4TX1MJ,4,3,POINT (-94.9835 29.19508)
2,219,1,2,19,1,550,316468003,4025,022612DCEB230FB9,None,...,119999,None,None,None,20060124,"US,US,graph,Chart 11322",US4TX1MJ,4,3,POINT (-94.9863 29.19827)
3,220,1,2,19,1,550,316468006,4025,022612DCEB260FB9,None,...,119999,None,None,None,20060124,"US,US,graph,Chart 11322",US4TX1MJ,4,3,POINT (-94.9824 29.19068)
4,8,1,2,19,1,550,522510788,12345,02261F24E1C43039,None,...,29999,None,None,None,20010206,"US,US,reprt,LNM 06/01",US5PR2JG,2,0,POINT (-65.62389 18.21222)



Verifying feature update status...
2025-09-22 20:51:29,955 - src.maritime_module.core.s57_data - INFO - Feature update status verification complete: {'VALID': np.int64(185969)}
Verification Summary:


,layer_name,enc_name,dsid_edition,dsid_update,feature_edition,feature_update,feature_count,edition_match,update_match,status
0,achare,US1GC09M,73,0,73,0,1,True,True,VALID
1,achare,US2AK20M,17,1,17,1,2,True,True,VALID
2,achare,US2AK5FM,28,0,28,0,1,True,True,VALID
3,achare,US2EC01M,31,0,31,0,3,True,True,VALID
4,achare,US3AK21M,21,3,21,3,1,True,True,VALID
...,...,...,...,...,...,...,...,...,...,...
185964,wrecks,US6OH05M,6,3,6,3,8,True,True,VALID
185965,wrecks,US6OH07M,4,0,4,0,1,True,True,VALID
185966,wrecks,US6OH1AD,1,0,1,0,1,True,True,VALID
185967,wrecks,US6SP10M,4,0,4,0,2,True,True,VALID


### 3.4. Test with Enterprise-Safe Parallel Processing

This test demonstrates using the `S57AdvancedConfig` to enable high-safety, read-only parallel processing, which can speed up the initial file discovery and preprocessing steps.

In [ ]:
# --- Step 1: Configure and Convert ---
parallel_schema = 's57_parallel_test'
print(f"--- Starting S57Advanced conversion with PARALLEL processing to schema: '{parallel_schema}' ---")

# Use the recommended high-safety configuration for parallel reads
high_safety_config = S57AdvancedConfig(
    enable_parallel_processing=True,
    parallel_read_only=True,
    parallel_db_writes=False,
    parallel_validation_level='strict',
    max_parallel_workers=2,
    enable_debug_logging=True
)

print("\nUsing configuration:")
print(high_safety_config.get_configuration_summary())

advanced_converter_parallel = S57Advanced(
    input_path=s57_data_dir,
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=parallel_schema,
    config=high_safety_config
)
advanced_converter_parallel.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=parallel_schema)
    summary = manager.get_enc_summary()
    print(f"Successfully created {len(summary)} ENCs in the parallel-processed schema.")
    display(summary.head())
except Exception as e:
    print(f"An error occurred during parallel processing verification: {e}")


## 4. S57Updater: Incremental Updates

`S57Updater` handles incremental updates to an existing S57Advanced database.

**Use Case**: Maintain a production ENC database with latest chart updates

**Update Modes**:
1. **Normal Update** (`update_from_location`): Apply incremental updates if available
2. **Force Update** (`force_update_from_location`): Replace specific ENCs entirely

**How It Works**:
- Compares local versions with update files
- Re-converts and replaces features for changed ENCs only
- Maintains transaction integrity (all-or-nothing updates)
- Provides detailed change summaries

**Caution**: Only use with PostGIS for now. File-based updates can cause corruption.


Update check:
Old Edition  -> New Edition |
US1EEZ1M 9:0  -> 9:0 |
US1GC09M 65:5 -> 71:1 |
US1PO02M 21:1 -> 21:1 |
US2WC12M 27:8 -> 27:12 |
US3CA52M 31:0 -> 31:7 |
US4CA60M 37:6 -> 38:2

### PostGIS Updater

In [ ]:
db_schema = 's57_advanced' # Use the schema created by S57Advanced
# Example S-57 file to update,
updater = S57Updater(output_format='postgis',
                     dest_conn=db_params,
                     schema=db_schema,
                     )
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()
display(summary_df)

### 4.2. SpatiaLite Updater

This test demonstrates the `S57Updater` functionality for a SpatiaLite database. The previous implementation failed with a `database disk image is malformed` error because it attempted to run the updater on a file that was being accessed by multiple, uncoordinated connections (OGR and SQLAlchemy).

The corrected test follows a robust, isolated process:

1.  **Create a fresh SpatiaLite database** from the initial ENC data (`s57_data_dir`). This provides a clean, known state to update and prevents file corruption.
2.  **Run the updater** on this new database using the update data (`s57_data_update_dir`).
3.  **Verify the results** by comparing the ENC versions before and after the update and checking for data integrity.

In [ ]:
sqlite_update_target_path = output_dir / 's57_advanced.sqlite'
# Example S-57 file to update,
updater = S57Updater(
     output_format='spatialite',
     dest_conn=str(sqlite_update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

### GPKG Updater

In [ ]:
update_target_path = output_dir / 's57_advanced.gpkg'
# Example S-57 file to update,
updater = S57Updater(
     output_format='gpkg',
     dest_conn=str(update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

## DeepTest

#### Setup

In [ ]:
from tests.core__real_data.deep_test_s57_workflow import *

db_params_test = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db_schema = 's57_deeptest'
test_output_path = project_root / 'test_output'

test_config = TestConfig(s57_data_root = s57_data_dir,
                         s57_update_root=s57_data_update_dir,
                         test_output_dir = test_output_path,
                         test_level = 3,

                         skip_postgis = False,
                         skip_updates = False,
                         cleanup_on_success = True,
                         postgis_config = db_params_test,
                         test_schema_name = db_schema,
                         clean_output = True,
                         exclude_extra_cols= ["geometry","geom", "wkb_geometry"],
)


print(f"\n--- Starting S57 Deep Test process with TEST CONFIG ---")
print(f"\n--- SettingUp directories:")
print(f"\nTest dataset at:       {test_config.s57_data_root}")
print(f"\nUpdate dataset at:     {test_config.s57_update_root}")
print(f"\nTest output directory: {test_config.test_output_dir}")
print(f"\n-----------------------------------")
post_conf = (f"Dababase: {db_params_test['dbname']} | Schema/Filename: {db_schema}") if  test_config.skip_postgis == False else ""
print(f"\nPostGIS tests:   {'❌' if test_config.skip_postgis == True else '✅'} {post_conf}")
print(f"\nUpdates process: {'❌' if test_config.skip_updates == True else '✅'}")


#### Load Dataset

In [2]:
try:
    print(f"\n🎉 Commence DeepTest !")
    tester = S57DeepTester(test_config)
except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

# Testing Update Readiness
if test_config.skip_updates == False:
    compare_df = tester.analyze_update_readiness()
    display(compare_df)



🎉 Commence DeepTest !


NameError: name 'logger' is not defined

In [ ]:
try:
    report = tester.run_comprehensive_test()

    print(f"\n🎉 DeepTest completed successfully!")
    print(f"📁 Results saved to: {test_config.test_output_dir}")

except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

In [2]:
import geopandas as gpd
import fiona

test_output_path = project_root / "tests" / "core__real_data" /"test_output"
# test_file = test_output_path / "s57_deeptest.gpkg"
test_file = test_output_path / "s57_deeptest.sqlite"
print(test_output_path)
print(test_file)
# --- 1. List all layers in the GeoPackage file ---
try:
    layer_names = fiona.listlayers(test_file)
    print(f"Layers found in '{test_file.name}':")
    for name in layer_names:
        print(f"- {name}")
except fiona.errors.DriverError as e:
    print(f"Error: Could not open the file '{test_file}'. Please check the path and ensure it's a valid GeoPackage file.")
    print(f"Details: {e}")
    # Exit if we can't open the file
    exit()

# --- 2. Select and read a specific layer ---
# Replace 'your_layer_name' with the name of the layer you want to open
# For example, if you have a layer named 'buildings', you would use:
# selected_layer_name = 'buildings'
selected_layer_name = 'lndmrk'

if selected_layer_name in layer_names:
    print(f"\nReading layer: '{selected_layer_name}'...")

    # Use geopandas.read_file with the layer parameter
    gdf = gpd.read_file(test_file, layer=selected_layer_name, engine="fiona")

    # --- 3. Work with your data ---
    print(f"Successfully loaded {len(gdf)} features from '{selected_layer_name}'.")

    # Print the first 5 rows of the GeoDataFrame
    print("\nFirst 5 rows of the layer:")
    test_df = gdf[gdf['colpat'].notna()]

    # Print information about the columns and data types
    print("\nLayer information:")
    gdf.info()

else:
    print(f"\nError: Layer '{selected_layer_name}' not found in the GeoPackage file.")
    print("Please choose one of the available layers listed above.")

/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/tests/core__real_data/test_output
/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/tests/core__real_data/test_output/s57_deeptest.sqlite
Layers found in 's57_deeptest.sqlite':
- isolatednode
- connectednode
- edge
- face
- admare
- airare
- bcnlat
- bcnspp
- bridge
- buisgl
- buaare
- boylat
- boysaw
- boyspp
- cblsub
- ctnare
- cgusta
- coalne
- conzne
- daymar
- depare
- depcnt
- dmpgrd
- exezne
- fogsig
- hrbfac
- lakare
- lndare
- lndelv
- lndrgn
- lndmrk
- lights
- magvar
- mipare
- obstrn
- pipsol
- rdocal
- rdosta
- rectrc
- resare
- rivers
- seaare
- sbdare
- slcons
- siltnk
- slogrd
- soundg
- tesare
- topmar
- tssbnd
- tsslpt
- tsezne
- uwtroc
- wattur
- wedklp
- wrecks
- m_covr
- m_npub
- m_nsys
- m_qual
- fshzne
- achare
- cblare
- cblohd
- damcon
- fairwy
- feryrt
- marcul
- morfac
- navlne
- ofsplf
- pilpnt
- pilbop
- pipare
- ponton
- prcare
- rtpbcn
- splare
- slotop
- unsare
- canals
- ctsare
- curent



In [3]:
test_df

,rcid,prim,grup,objl,rver,agen,fidn,fids,lnam,lnam_refs,...,scamin,txtdsc,recdat,recind,sordat,sorind,dsid_dsnm,dsid_edtn,dsid_updn,geometry
79,1948,1,2,74,1,550,28063069,50,022601AC355D0032,"[022601AC355F0032, 022601AC355E0032, 022601AC3...",...,9999999,None,None,None,20111201,"US,US,graph,Chart 11524",US1GC09M,65,5,POINT (-79.84328 32.75795)
81,1950,1,2,74,1,550,450842981,6465,02261ADF51651941,"[02261E441DC01941, 022626A0CF061941]",...,9999999,None,None,None,20151104,"US,US,reprt,L-1615/2015, NGA light list #325.00",US1GC09M,65,5,POINT (-78.88337 22.68496)
91,1960,1,2,74,1,550,35445370,50,0226021CDA7A0032,[0226021CC8020032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-83.2159 22.98921)
104,1973,1,2,74,1,550,35439108,50,0226021CC2040032,[0226021CC91C0032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-84.02367 22.71648)
112,1981,1,2,74,1,550,35440243,50,0226021CC6730032,[0226021CD0630032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-81.0399 21.66121)
115,1984,1,2,74,1,550,253936398,4044,02260F22C30E0FCC,"[02260F22BF4F0FCC, 02260F22BFD90FCC]",...,9999999,None,None,None,200303,"US,US,graph,Chart 411",US1GC09M,65,5,POINT (-89.69113 22.38822)
142,2011,1,2,74,1,550,8745907,50,0226008573B30032,[02260085734D0032],...,9999999,None,None,None,20030121,"US,US,reprt,7thCGD,LNM 03/03",US1GC09M,65,5,POINT (-80.54345 28.46035)
144,2013,1,2,74,1,550,253936611,4044,02260F22C3E30FCC,[02260F22C01C0FCC],...,9999999,None,None,None,200303,"US,US,graph,Chart 411",US1GC09M,65,5,POINT (-76.19634 17.91273)
145,2014,1,2,74,1,550,-1145870482,12345,0226BBB3676E3039,"[0226000006FD0001, 02263377A9D31941]",...,9999999,None,None,None,2015,"US,US,reprt,NGA light List #12676",US1GC09M,65,5,POINT (-80.48141 23.20533)
151,2020,1,2,74,1,550,368797181,6465,022615FB65FD1941,[022620822C7A1941],...,9999999,None,None,None,20151104,"US,US,reprt,L-1615/2015, NGA light list #11800",US1GC09M,65,5,POINT (-76.96487 26.53864)
